In [1]:
import dash
from dash import dcc
from dash import html
from dash import dash_table as dt
from dash.dependencies import Input, Output
import plotly.express as px
import webbrowser
import pandas as pd
import numpy as np

In [2]:
#Getting spreadsheet data
sheetid = '1u1c_ZiucAge9VDjxRD6hL8pnLWCgxZPBYnYJYbuqLH4'
sheetname = 'Dagger Weapon Info Database'
sheetname = sheetname.replace(" ","%20") #unsure what this does maybe spaces replacement?
gsheet_url = 'https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}'.format(sheetid,sheetname)

#Data into a dataframe
df = pd.read_csv(gsheet_url)
#df.drop(index=0,inplace=True)
df.reset_index(drop=True,inplace=True)

#Cleaning the data format and entries
#Possible to adjust spreadsheet such that headers are more readable to this so it's not so manual but meh
columns = ['Type','Weapons','Attack','Crit','Blade Instinct','Special Mod','Finesse','Attack 1 MRM','Attack 2 MRM','Attack 3 MRM','Attack 4 MRM','Attack 1 Dmg',
          'Attack 2 Dmg','Attack 3 Dmg','Attack 4 Dmg','Attack 1 %','Attack 2 %','Attack 3 %','Attack 4 %','Asp MRM','Adder MRM','Sidewinder MRM','Vipers Nest MRM',
           'Asp Dmg','Adder Dmg','Sidewinder Dmg','Vipers Nest Dmg','Passive DoT Rank','Skill Attack','Skill Type','Skill Damage','Charge Time','Use Time','Duration - Use Time','Modifier','Skill DoT Cons.','Skill DoT Lvl Slope','Tot Ticks','Tot Time']
for colind,col in enumerate(df.columns):
    df.rename(columns={col:columns[colind]}, inplace=True)
#print(df['Type'])
for rowind,row in enumerate(df['Type']):
    if not pd.isna(row):
        curr_type = row
    else:
        #df['Type'][rowind] = curr_type
        df.loc[rowind,'Type'] = curr_type

pd.set_option("display.max_columns",None)


In [3]:
#Function to simulate the game
def DAGGEROPTDPS(dagType,finesse, damage,skillCooldownVals, skillType, skillCooldown,useTime,duration,modifier,
                dotConstant,dotSlope,dotTicks,dotTime,level,HP, enemyType, players,poisonRank,finalTime,skip2 = False,skip3 = False,
                skip4=False, time = 0,currDmg = 0, backstabs = True,woundVals = [2,1,0,0]):
    
    skillCooldownVals = np.array(skillCooldownVals)
    skillCooldownVals/=finesse
    
    #Define some useful variables
    
    #Variables describing a skill's damage over time
    dotTimer = 0
    tickTimer = 0
    tickDmg = dotConstant + dotSlope * level
    tickDmgTot = 0 #needed to track later
    
    #Track the attacks that are done
    attacks = [0,0,0,0,0]
    buffedAttacks = [0,0,0,0,0]
    
    #Wound information for proper tracking
    woundCooldown = woundVals[0]
    nextWoundInterval = woundVals[1]
    woundDmgTot = woundVals[2]
    woundAttacks = woundVals[3]
    
    #Poison information trackers
    poisonTimer = 0 #start at 0 then on hit set to 4, if was 0 then poison occurs earlier
    nextPoisonInterval = 0
    poisonDmgTot = 0
    poisonAttackIndex = 0 #marks what attack was done for reference
    
    #Form the animation time lists of attacks
    if "Dagger" in dagType:
        attackTimes = [.75,.75,1,2.4,useTime]
        if backstabs:
            damage = list(np.array(damage)*1.25) #Ups damage values by 25% since assuming backstabs
            #Apply backstabs for bestestestestest damage
    elif "Throwing" in dagType:
        attackTimes = [.67,.63,.94,1.43,useTime]
    
    ####PRELIMINARIES DONE
    
    ####BEGIN ATTACK SIMULATION
    
    ######Notes
    ##Break attacks use cooldowns between 0 and 1 to represent the portion of bar left to fill
        ##i.e. 0 means it is ready to use, 1 is completely empty
    ##Cooldown skill use time between 0 and max cooldown
    
    ##Adjust start if using buffing skill
    if duration > 0:
        #This occurs for weapons with a "buff" skill that lasts for some time
        #Assume we just used the skill as this will be optimal to be buffed up at the start

        if "Break" == skillType:
            currSkillCooldown = 1
        elif "Cooldown" == skillType:
            currSkillCooldown = finesse * skillCooldown - useTime
        
        durationTimer = duration #Already accounts for the usetime in database
        attacks[4] += 1 #technically used it at the start
    else:
        #Not a buff skill on the sword
        #Start with skill and no buffs
        currSkillCooldown = 0
        durationTimer = 0
        
        
    #More variables
    currAttInd = 0 #What stage in combos sequence we are at
    #Determine the values for the combo based on what isn't skipped
    attackSequence = [0,1,2,3]
    if skip2:
        attackSequence.remove(1)
    if skip3:
        attackSequence.remove(2)
    if skip4:
        attackSequence.remove(3)
    currAtt = attackSequence[currAttInd] #What attack in combos we are at

    ##Basically just a shorthand for later usage
    
    #####START SIM
    
    #print("Start")
    #print("Curr Dmg:",currDmg,"  time:",time)
    while (time < finalTime):
        attackTimeTracker = time #For DoT if needed
        
        #NO SKILL OR SKILL ON COOLDOWN
        if (pd.isna(skillType) or (currSkillCooldown > 0)):
            #print("Normal")
            #Adjust combo sequence marker
            attackNum = currAtt
            if currAtt >= attackSequence[-1]:
                #If using the last in combo
                currAttInd = 0
                currAtt = attackSequence[0]
            else:
                currAttInd += 1
                currAtt = attackSequence[currAttInd]
                
            #Now basic stuff for attacks
            time+=attackTimes[attackNum] #increment time from using attack
            #If we're buffed provide buff damage
            if (durationTimer > 0):
                buffedAttacks[attackNum] += 1
                currDmg += modifier*damage[attackNum]
            else:
                attacks[attackNum] += 1
                currDmg += damage[attackNum]
            #reduce the buff time duration by attack time
            durationTimer -= attackTimes[attackNum]
            if ("Cooldown" == skillType):
                currSkillCooldown -= attackTimes[attackNum]
            elif "Break" == skillType:
                currSkillCooldown -= skillCooldownVals[attackNum]
            #Wound applications
            #Count an attack done, adjust cooldowns
            woundAttacks += 1
            if woundAttacks > 0:
                woundCooldown -= attackTimes[attackNum]
                
            #mark attack for poison
            poisonAttackIndex = attackNum
        #SKILL OFF COOLDOWN, USABLE
        elif currSkillCooldown <= 0:
            ##FIRST GET THE DPS IF IT IS USED
            #THEN USE THE HIGHEST ONE AND ADJUST VARIABLES FROM USING IT
            
            ##DPS if we used these
            attDPS = (currDmg+damage[currAtt])/(time+attackTimes[currAtt])
            
            #Get skill DPS
            #First get it for buff skills
            if duration > 0 and durationTimer <= 0: #I.e. buff has a duration and we're not currently under its effects 
                #Assume we use it and recurse back
                if time+attackTimes[4]+duration > finalTime:
                    a,b = DAGGEROPTDPS(dagType,finesse, damage,skillCooldownVals, skillType, skillCooldown,useTime,duration,modifier,dotConstant,
                                                      dotSlope,dotTicks,dotTime,level,HP,enemyType,players,poisonRank,finalTime,skip2,skip3,skip4, time+attackTimes[4],currDmg,backstabs,
                                                      [woundCooldown,nextWoundInterval,woundDmgTot,woundAttacks])
                    c = a + b
                    skillDmg = finalTime*c
                    skillDPS = skillDmg/(finalTime)
                    skillFinalTime = finalTime
                else:
                    a,b = DAGGEROPTDPS(dagType,finesse, damage,skillCooldownVals, skillType, skillCooldown,useTime,duration,modifier,dotConstant,dotSlope,dotTicks,dotTime,
                                            level,HP,enemyType,players,poisonRank,time+attackTimes[4]+duration,skip2,skip3,skip4, time+attackTimes[4],currDmg,backstabs,
                                            [woundCooldown,nextWoundInterval,woundDmgTot,woundAttacks])
                    c = a + b
                    skillDmg = (time+attackTimes[4]+duration)*c
                    skillDPS = skillDmg/(time+attackTimes[4]+duration)
                    skillFinalTime = time+attackTimes[4]+duration
            #Now for non-buff skills
            #Isn't that so much easier!
            else:
                skillDPS = (currDmg + damage[4])/(time+attackTimes[4])
                
            ###Determine what to do for best greedy
            
            #Between attack dps and skill dps
            if skillDPS > attDPS and duration <= 0:
                #print("Skill")
                #Actually use the skill and reset combo marker
                attackNum = 4
                currAttInd = 0
                currAtt = attackSequence[currAttInd]
                #Deal with DoT effects
                dotTimer = dotTime
                if dotTicks != 0:
                    tickTimer = (dotTime/dotTicks)
                    
                #mark attack for poison
                poisonAttackIndex = 4
            else:
                #print("Normal")
                #Combos win, adjust combo sequence
                attackNum = currAtt
                if currAtt == attackSequence[-1]:
                    #If using the last in combo
                    currAttInd = 0
                    currAtt = attackSequence[0]
                else:
                    currAttInd += 1
                    currAtt = attackSequence[currAttInd]
                    
                #mark attack for poison
                poisonAttackIndex = attackNum
                
            #Now stuff for either one
            time+=attackTimes[attackNum] #increment time from using attack
            #If we're buffed provide buff damage
            if (durationTimer > 0):
                buffedAttacks[attackNum] += 1
                currDmg += modifier*damage[attackNum]
            else:
                attacks[attackNum] += 1
                currDmg += damage[attackNum]
            #reduce the buff time duration by attack time
            durationTimer -= attackTimes[attackNum]
            
            if attackNum != 4:
                #Didn't use skill, adjust normally
                #Adjust skill cooldowns from attack
                if ("Cooldown" == skillType):
                    currSkillCooldown -= attackTimes[attackNum]
                elif "Break" == skillType:
                    currSkillCooldown -= skillCooldownVals[attackNum]
            elif attackNum == 4:
                #Did use the skill
                #Adjust accordingly
                if ("Cooldown" == skillType):
                    currSkillCooldown = skillCooldown*finesse - attackTimes[4]
                elif "Break" == skillType:
                    if duration > 0:
                        durationTimer = duration
                    currSkillCooldown = 1
                    
            woundAttacks += 1
            #Wound adjustments
            if woundAttacks > 0:
                woundCooldown -= attackTimes[attackNum]

        #Ok we've chosen our attack/skill for this loop
        #print(woundCooldown)
        #Check the wound ticks done during this attack and add damage accordingly and adjust timers
        while(woundCooldown <= 0):#Wound needs applied, went off cooldown so some wound damage occurred.
            #Add wound damage
            #Translate woundAttacks to woundStacks for slightly better calculation
            if enemyType == "Enemy":
                if (woundAttacks == 1):
                    woundStacks = 9
                else:
                    woundStacks = (woundAttacks - 1)*8 + 9 #starts at 9, each attack after adds 8
            elif enemyType == "Boss":
                woundStacks = woundAttacks * 3
            #Now make sure the stacks don't exceed the limit of 200
            if (woundStacks > 200):
                woundStacks = 200
            if enemyType == "Enemy":
                woundDmgTot += 0.001*woundStacks*HP
            elif enemyType == "Boss":
                woundDmgTot += 0.0005*woundStacks*HP
                
            #print(woundDmgTot,woundStacks)
            #Reset cooldown
            woundCooldown += nextWoundInterval
            #Adjust interval
            if (nextWoundInterval == 1):
                nextWoundInterval = 2
            else:
                nextWoundInterval = 1
            
            
        #Now do DoT ticks for the amount of time spent
        #Doing this particular attack
        #print("PreTick")
        ##print("Curr Dmg:",currDmg,"  time:",time)
        #print(dotTimer,tickTimer)
        if time < finalTime:
            dotEndTime = time
        else:
            dotEndTime = finalTime
        if (dotTimer != 0):
            #Still time on total DoT
            dotTimer -= (dotEndTime-attackTimeTracker)
            tickTimer -= (dotEndTime-attackTimeTracker)
            #Perform damage if decide the time has passed on the next tick
            if (tickTimer <= 0 and dotTicks != 0): ##evaluated we need to do a tick of damage
                tickDmgTot += tickDmg
                tickTimer += dotTime/dotTicks #adjust to next tick time
            if (dotTimer < 0): #Implies gone past time of duration of DoT, so set to 0
                dotTimer = 0
                tickTimer = 0
        #print("PostTick")
        #print("Curr Dmg:",currDmg,"  time:",time)
        
        
        #Poison damage calculations
        #Poison DoT
        #Adjust poison timer
        if (poisonRank > 0):
            if (poisonAttackIndex != 4):
                #Then we did a normal attack, reset timer otherwise assume it's ongoing
                poisonTimer = 4
            #begin calculating timer differences after whatever attack just happened
            if (attackTimes[poisonAttackIndex] > nextPoisonInterval and poisonTimer > 0):
                #then some tick occurs during this animation of attack
                #get the exact moment it would happen relative to poisonTimer and check what damage it should do based on time left on timer
                #Well it would happen in nextPoisonInterval seconds left until it reaches 0
                poisonTimer -= nextPoisonInterval
                if (poisonTimer < 0):
                    #Then we lost the poison before we hit the next interval so nothing happens
                    poisonTimer = 0
                    nextPoisonInterval = 0.5
                else:
                    #Poison ticks and timer still around
                    #Ok now we need to apply the tick based on where timer is right now
                    if (poisonTimer >= 3):
                        if (enemyType == "Enemy"):
                            #1% hp tick
                            poisonDmgTot += HP*0.01
                        else:
                            #0.5% hp tick
                            poisonDmgTot += HP*0.005
                    elif (poisonTimer >= 3):
                        if (enemyType == "Enemy"):
                            #1.5% hp tick
                            poisonDmgTot += HP*0.015
                        else:
                            #0.67% hp tick
                            poisonDmgTot += HP*0.00666
                    elif poisonTimer > 0:
                        if (enemyType == "Enemy"):
                            #2% hp tick
                            poisonDmgTot += HP*0.02
                        else:
                            #1% hp tick
                            poisonDmgTot += HP*0.01

                    #decrement times fully now
                    nextPoisonInterval -= attackTimes[poisonAttackIndex] #finish concept of interval decrementing from animation
                    nextPoisonInterval += 1 #Then we found it occurred and should take 1 second to next
                    poisonTimer -= (attackTimes[poisonAttackIndex] - nextPoisonInterval)
            else:
                #no tick occurs here so simply dec the timers fully
                poisonTimer -= attackTimes[poisonAttackIndex]
                nextPoisonInterval -= attackTimes[poisonAttackIndex]

            #Now check if the poisonTimer dissappeared finally and adjust values accordingly, otherwise timer still active and return to top of loop
            if (poisonTimer < 0):
                poisonTimer = 0
                nextPoisonInterval = 0.5
    

    if "Throwing" in dagType:
        woundDmgTot = 0 #Throwing daggers can't do wound attacks
    
    woundDPS = woundDmgTot/players/finalTime #Invididual contribution
    comboDPS = currDmg/finalTime
    skillTickDPS = tickDmgTot/finalTime
    poisonDPS = poisonDmgTot/finalTime
    tickDPS = woundDPS + skillTickDPS + poisonDPS
    #OUTSIDE OF THE WHILE LOOP NOW
        #print("Curr Dmg:",currDmg,"  time:",time)
    #print(woundDPS,comboDPS)
    return tickDPS,comboDPS

In [4]:
def woundTTK(players,enemyType):
    #Wound information for proper tracking
    woundCooldown = 2
    nextWoundInterval = 1
    woundAttacks = 0
    
    attackTimes = [.75,.75,1,2.4]
    attackNum = 0
    dead = False
    HP = 1
    time = 0
    hits = 0
    while not dead:
        if HP <= 0:
            dead = True
            continue
        else:
            #Do an attack and check if wound needs applied
            hits += 1 #Increment a hit
            time += attackTimes[attackNum] #Increment time from hit
            attackNum += 1 #increment combo index
            if attackNum >= 4:
                attackNum = 0 #reset combo index if done with combo
                
            woundCooldown -= attackTimes[attackNum] #decrement wound tick timer by attack time
            
            #Determine if wound tick needs applied now
            while(woundCooldown <= 0):#Wound needs applied, went off cooldown so some wound damage occurred.
                #Add wound damage
                #Translate woundAttacks to woundStacks for slightly better calculation
                if enemyType == "Enemy":
                    woundStacks = hits * 8
                elif enemyType == "Boss":
                    woundStacks = hits * 3
                    
                woundStacks *= players #Adjust total stacks by amount of players collectively also hitting
                
                #Now make sure the stacks don't exceed the limit of 200
                if (woundStacks > 200):
                    woundStacks = 200
                #Reduce HP by percentage HP
                if enemyType == "Enemy":
                    HP -= 0.001*woundStacks
                elif enemyType == "Boss":
                    HP -= 0.0005*woundStacks

                #print(woundDmgTot,woundStacks)
                #Reset cooldown
                woundCooldown += nextWoundInterval
                #Adjust interval
                if (nextWoundInterval == 1):
                    nextWoundInterval = 2
                else:
                    nextWoundInterval = 1
    return time

In [5]:
#Simple function to return the weapon multiplier
def modifiers(attack,crit, BI, SM):
    return 1.5*(1+.03*attack)*crit*BI*SM

In [6]:
#Helper function to get the total sword DPS
#Calling the simulation * modifiers adding in poison
def extractDPS(df,rownum,level,HP,enemyType,players,finalTime,backstabs = True,skip2 = False, skip3=False,skip4=False):
    m = modifiers(df["Attack"][rownum],df["Crit"][rownum],df["Blade Instinct"][rownum],df["Special Mod"][rownum])
    tickDPS,comboDPS = DAGGEROPTDPS(df["Type"][rownum],df["Finesse"][rownum],[df["Attack 1 Dmg"][rownum],df["Attack 2 Dmg"][rownum],
                    df["Attack 3 Dmg"][rownum],df["Attack 4 Dmg"][rownum],df["Skill Damage"][rownum]],
                    [df["Attack 1 %"][rownum],df["Attack 2 %"][rownum],df["Attack 3 %"][rownum],df["Attack 4 %"][rownum],0],
                    df["Skill Type"][rownum],df["Charge Time"][rownum], df["Use Time"][rownum], df["Duration - Use Time"][rownum],
                    df["Modifier"][rownum],df["Skill DoT Cons."][rownum],df["Skill DoT Lvl Slope"][rownum],df["Tot Ticks"][rownum],df["Tot Time"][rownum],level,HP,enemyType,players,df["Passive DoT Rank"][rownum],
                    finalTime,skip2,skip3,skip4,0,0,backstabs,[2,1,0,0])
    finalDPS = comboDPS*m + tickDPS
    return finalDPS

In [7]:
#Function to extract DPS with times from 1 to 30 second values
#Returns dataframe
def timelineDPS(df,level,HP,enemyType,skips,players = 1,backstab = "True"):
    #Get backstab value in better format
    if backstab == "True":
        backstab = True
    else:
        backstab = False
    #Get skipvals
    bool2 = False
    bool3 = False
    bool4 = False
    for skip in skips:
        if "2" in skip:
            bool2 = True
        elif "3" in skip:
            bool3 = True
        elif "4" in skip:
            bool4 = True
    weaponNames = df["Weapons"].to_list()
    #This is by column data, need to contrust dictionary data type passed to dataframe
    d = dict([])
    for rownum in range(len(df)):
        weaponName = df["Weapons"][rownum]
        dpsList = [extractDPS(df,rownum,level,HP,enemyType,players,time,backstab,skip2=bool2,skip3=bool3,skip4=bool4) for time in range(1,32)]
        d[weaponName] = dpsList
    timeDF = pd.DataFrame(d)
    return timeDF

In [8]:
#The dashboard application
#Presents two of the same graph and options
#So the user may compare differing graphs of DPS vs time
#Baseline Graph

app = dash.Dash(__name__)
app.layout = html.Div([
    ##LEFT SIDE GRAPH
    html.Div([
        ##Graph
        dcc.Graph(id='DPS vs Time 1'),
        #Sword Picker
        html.Div([
            "Daggers To Graph",
            dcc.Dropdown(
                id='dagger-dropdown 1', clearable=False,
                value='Top 10', options=[{'label':y, 'value':y} for x in [["Top 5","Top 10","All"],timelineDPS(df,1,10000,"Boss",[],1,True).columns.sort_values()] for y in x]
                )
        ],style = {'padding-bottom':'10px','padding-top':'10px'}),
        #Skip values picker
        html.Div([
            "Combo Attacks To Skip",
            dcc.Checklist(
                id='skip-list 1',
                value=[], options=[{'label':'Skip 2nd', 'value':'Skip 2nd'},{'label':'Skip 3rd', 'value':'Skip 3rd'},{'label':'Skip 4th', 'value':'Skip 4th'}]
                )
        ],style = {'padding-bottom':'10px','padding-top':'10px'})
    ], style = {'width': '49%', 'height' : '33%','display': 'inline-block'}),

    #RIGHT SIDE GRAPH
    html.Div([
        dcc.Graph(id='DPS vs Time 2'),
        #Weapon Picker
        html.Div([
            "Daggers To Graph",
            dcc.Dropdown(
                id='dagger-dropdown 2', clearable=False,
                value='Top 10', options=[{'label':y, 'value':y} for x in [["Top 5","Top 10","All"],timelineDPS(df,1,10000,"Boss",[],1,True).columns.sort_values()] for y in x]
                )
        ],style = {'padding-bottom':'10px','padding-top':'10px'}),
        #Skip values picker
        html.Div([
            "Combo Attacks To Skip",
            dcc.Checklist(
                id='skip-list 2',
                value=[], options=[{'label':'Skip 2nd', 'value':'Skip 2nd'},{'label':'Skip 3rd', 'value':'Skip 3rd'},{'label':'Skip 4th', 'value':'Skip 4th'}]
                )
        ],style = {'padding-bottom':'10px','padding-top':'10px'})
    ],style={'width': '49%', 'height' : '33%','float': 'right','display': 'inline-block'}),

    #ENEMY LEVEL DROPDOWN
    html.Div([
        "Enemy Level",
        dcc.Dropdown(
            id='level-dropdown', clearable=False,
            value='50', options=[{'label':y, 'value':y} for y in range(1,51)]
        )
    ],style = {'padding-bottom':'10px','padding-top':'10px'}),
    #BACKSTABS DROPDOWN
    html.Div([
        "Backstabbing?",
        dcc.Dropdown(
            id='backstab-dropdown', clearable=False,
            value='True', options=[{'label':y, 'value':y} for y in ['True','False']]
        )
    ],style = {'padding-bottom':'10px','padding-top':'10px'}),
    #ENEMY TYPE DROPDOWN
    html.Div([
        "Enemy Type",
        dcc.Dropdown(
            id='enemy-dropdown', clearable=False,
            value='Boss', options=[{'label':y, 'value':y} for y in ['Enemy','Boss']]
        )
    ],style = {'padding-bottom':'10px','padding-top':'10px'}),
    #HP SLIDER
    html.Div([
        "Enemy HP",
        dcc.Slider(0,
        200000,
        value=10000,
        #marks = dict(enumerate([str(i) for i in range(0,10001,5000)])),
        marks={i: '{}'.format(i) for i in range(0,200001,5000)},
        id = 'enemy-HP-slider',
        tooltip={"placement": "bottom", "always_visible": True}
        )
    ],style = {'padding-bottom':'20px','padding-top':'10px'}),
    ##TIME SLIDER
    html.Div([
        "Graph Time Endpoints",
        dcc.RangeSlider(
        id = 'time-range',
        min = 1,
        max = 30,
        step = 1,
        value = [1,30],
        marks = dict(enumerate([str(i) for i in range(0,31)]))
        )
    ],style = {'padding-bottom':'20px','padding-top':'10px'}),

    #LEFT AVG TABLE
    html.Div([
        dt.DataTable(
            id='avgtbll',
            data = [],
            columns = [{"name":"Weapon","id":"sword"},{"name":"Left Graph Avg DPS","id":"mean"}]
        )
    ],style = {'width': '49%', 'display': 'inline-block'}),

    #RIGHT AVG TABLE
    html.Div([
        dt.DataTable(
            id='avgtblr',
            data = [],
            columns = [{"name":"Weapon","id":"sword"},{"name":"Right Graph Avg DPS","id":"mean"}]
        )
    ],style = {'width': '49%', 'float':'right','display': 'inline-block','padding-top':'15px'}),
])


#OUTPUTS
#Graphs

@app.callback(
    [Output('DPS vs Time 1', 'figure'),Output('DPS vs Time 2', 'figure')],
    [Input("level-dropdown", "value"),
    Input("enemy-HP-slider","value"),
    Input("enemy-dropdown","value"),
    Input("backstab-dropdown", "value"),
    Input('time-range','value'),
    Input('dagger-dropdown 1','value'),
    Input('dagger-dropdown 2','value'),
    Input('skip-list 1','value'),
    Input('skip-list 2','value')])


def update_DPS_Fig(level,HP,enemyType,backstab,time_range,daggers,daggers2,skips,skips2):
    #Dataframe
    daggerInputs = [daggers,daggers2]
    skipInputs = [skips,skips2]
    timeDFList = [timelineDPS(df,int(level),HP,enemyType,skip,1,backstab) for skip in skipInputs] #1 is the number of players here which doesn't matter since it's divided out anyways
    #Time range 1 to 30
    graphs = []
    for listIndex in range(0,2):
        xtime = [i for i in range(1,len(timeDFList[listIndex])+1)]
        #Find the y columns to use
        if daggerInputs[listIndex] == "Top 10":
            avg = timeDFList[listIndex].mean().sort_values(ascending=False)
            ycols = list(avg[0:10].index)
        elif daggerInputs[listIndex] == "Top 5":
            avg = timeDFList[listIndex].mean().sort_values(ascending=False)
            ycols = list(avg[0:5].index)
        elif daggerInputs[listIndex] == "All":
            ycols = list(timeDFList[listIndex].columns.sort_values())
        else:
            ycols = [str(daggerInputs[listIndex])]
        graph = px.line(timeDFList[listIndex],x=xtime,y=ycols,labels={"value":"DPS","x":"Time (s)","variable":"Dagger"})
        graph.update_xaxes(range = time_range)
        rangemin = time_range[0]
        rangemax = time_range[1]
        #Those define the rows to check through
        ymax = 0
        ymin = 999999
        #Since time_range is fixed up by 1 to help fit the plot
        #better, muust dec the min and max down 1
        #when searching through the rows in the df
        for rownum in range(rangemin-1,rangemax):
            for weapon in ycols:
                if ymax < timeDFList[listIndex][weapon][rownum]:
                    ymax = timeDFList[listIndex][weapon][rownum]
                if timeDFList[listIndex][weapon][rownum] < ymin:
                    ymin = timeDFList[listIndex][weapon][rownum]
        #Should have the max value in the time frame selected for all weapons
        #Update graph
        graph.update_yaxes(range = [ymin,ymax])
        graphs.append(graph)
    return graphs

#Tables
@app.callback(
    [Output('avgtbll', 'data'),Output('avgtblr', 'data')],
    [Input("level-dropdown", "value"),
    Input("enemy-HP-slider","value"),
    Input("enemy-dropdown","value"),
    Input("backstab-dropdown", "value"),
    Input('time-range','value'),
    Input('dagger-dropdown 1','value'),
    Input('dagger-dropdown 2','value'),
    Input('skip-list 1','value'),
    Input('skip-list 2','value')])

def update_table(level,HP,enemyType,backstab,time_range,daggers,daggers2,skips,skips2):
    #Dataframe
    daggerInputs = [daggers,daggers2]
    skipInputs = [skips,skips2]
    timeDFList = [timelineDPS(df,int(level),HP,enemyType,skip,1,backstab) for skip in skipInputs]
    #Find the y columns to use
    tables=[]
    for listIndex in range(0,2):
        #Find the y columns to use
        if daggerInputs[listIndex] == "Top 10":
            avg = timeDFList[listIndex].mean().sort_values(ascending=False)
            ycols = list(avg[0:10].index)
        elif daggerInputs[listIndex] == "Top 5":
            avg = timeDFList[listIndex].mean().sort_values(ascending=False)
            ycols = list(avg[0:5].index)
        elif daggerInputs[listIndex] == "All":
            ycols = list(timeDFList[listIndex].columns.sort_values())
        else:
            ycols = [str(daggerInputs[listIndex])]
        avg = []
        for col in ycols:
            avg.append(timeDFList[listIndex][col].mean())

        subsetDF = timeDFList[listIndex][ycols]
        avgs = list(subsetDF.mean())
        avgDF = pd.DataFrame(data = [avgs],columns = [ycols],index = ['mean'])
        avgDF = avgDF.transpose()
        avgDF['sword'] = avgDF.index.to_numpy()
        tables.append(avgDF.to_dict(orient='records'))
    return tables

##Run it
if __name__ == '__main__':
    port = 8050
    webbrowser.open_new("http://localhost:{}".format(port))
    app.run_server(port=port)